### Si bien este laboratorio es corto, se pide trabajar de forma lenta con los estudiantes, ya que programación dinámica estocástica es un tema que les cuesta mucho. Por lo tanto ideal ir viendo paso a paso.



Un papá debe decidir todos los días hábiles de la semana (lunes a viernes) si lleva, o no, a su hija al jardín. 

Si la niña va sana al jardín, la probabilidad de que amanezca enferma al día siguiente es 0.3 y si la niña va enferma al jardín, la probabilidad que siga enferma al día siguiente es 0.8. Cada día que amanece enferma, la familia debe gastar \$5.000 en medicamentos (incluso los fines de semana). 

En caso de que el papá elija no llevar a su hija al jardín (independiente de si está enferma o sana), debe pagar a una persona cuidadora un costo de \$10.000 por el día. Cuando la niña se queda en casa, con seguridad amanece sana al día siguiente (independiente de si estaba sana o enferma).

Se desea estudiar la situación para los próximos K días de la semana, siendo el K-ésimo el día viernes, sabiendo que el sábado la familia debe irse de viaje. En caso que la hija amanezca enferma el día sábado, la familia no podrá viajar lo que representa un costo de \$20.000 (que no incluye los medicamentos).

Para esto se ha formulado el siguiente modelo de programación dínamica

**Etapas**

$K$, una por cada día.

**Variables de decisión**:

$
X_n= \left\{\begin{array}{cl}
	1& \text{si la niña asiste al jardín el día $n$} \\
	0& \text{si la niña no asiste al jardín el día $n$} \\
\end{array}\right.
$ 

**Variables de estado**: 

$X_n= \left\{\begin{array}{cl}
	1& \text{si la niña amanece sana el día $n$} \\
	0& \text{si la niña amanece enferma el día $n$} \\
\end{array}\right.$ 

**Reglas de transformación** 

$S_{n+1}=\left\{\begin{array}{ccl}
	1& \text{si} & X_n=0 \\
	1& \text{si} & X_n=1, S_n=1, \text{ con } p=0.3 \\
	0& \text{si} & X_n=1, S_n=1, \text{ con } p=0.7 \\
	1& \text{si} & X_n=1, S_n=0, \text{ con } p=0.2 \\
	0& \text{si} & X_n=1, S_n=0, \text{ con } p=0.8 \\    
\end{array}\right.$ 

**Función de recompensa**

$R_n(S_n,X_n) = 5000(1-S_n)+10000(1-X_n)$

**Función recursiva**

$f_n(S_n,X_n)=\mathbb{E}(R_n(S_n,X_n)+f_{n+1}^*(S_{n+1}))$, para todo $n$.

**Función objetivo** 

$\displaystyle f_n^*(S_n)=\max_{X_n} (f_n(S_n,X_n))$ para todo $n$.

**Condiciones**

$f_{K+1}^*(S_{K+1})=25000(1-S_{K+1})$

**Restricciones**

$X_n, S_n \geq 0$, enteras, para todo $n$.



En primer lugar necesitamos definir tres funciones: la función de transformación (función de actualización de la variable de estado), función de recompensa y la función recursiva.

Para las etapas usaremos un arreglo de tres dimensiones, la primera dimensión corresponderá a las distinas etapas, la segunda a los valores de la variable de estado y la tercera a los valores de la variable de decisión. Como en este caso es posible asignar cualquier cantidad de equipos médicos a un país (respetando las restricciones) las filas van de $0$ a $R$, al igual que las columnas.

Asignaremos el valor -1 para inicializar las matrices de las etapas, a fin de poder determinar aquellas celdas infactibles.

In [57]:
def funcion_recompensa(S,X):
    R = 5000*(1-S)+10000*(1-X)
    return R

In [60]:
import numpy as np
def jardin(T): # T será la cantidad de días.
    etapas = -1*np.ones((T,2,2)) # generamos las T etapas, cada una con 2 filas y 2 columnas ya que hay dos estados posibles.
    #generamos la última tabla, la condición de borde
    mejores = np.zeros((2,T+1)) # generamos la matriz para los óptimos.
    mejores[0,T] = 25000
    mejores[1,T] = 0
    for k in range(T-1,-1,-1):
        for i in range(2):
            for j in range(2):
                if j==0:
                    etapas[k,i,j]=funcion_recompensa(i,j)+mejores[1,k+1] # calculamos el valor actual y le sumamos la parte recursiva
                elif j==1 and i==0:
                    etapas[k,i,j]=funcion_recompensa(i,j)+0.8*mejores[0,k+1]+0.2*mejores[1,k+1] 
                elif j==1 and i==1:
                    etapas[k,i,j]=funcion_recompensa(i,j)+0.3*mejores[0,k+1]+0.7*mejores[1,k+1] 
                    # calculamos el valor actual y le sumamos la parte recursiva
                    # de esta forma, calculamos la cantidad de equpos sobrantes, correspondientes a los
                # que tenemos menos los que asignamos.
            mejores[i,k]=np.min(etapas[k,i,:]) # para cada fila, determinamos el mejor valor posible.
    return etapas,mejores

In [83]:
a,b = jardin(3)
print(a)
print(b)

[[[24750. 21750.]
  [19750. 12375.]]

 [[22500. 18500.]
  [17500.  9750.]]

 [[15000. 25000.]
  [10000.  7500.]]]
[[21750. 18500. 15000. 25000.]
 [12375.  9750.  7500.     0.]]


In [87]:
# Ahora generaremos una metodología para poder determinar las decisiones a tomar.
def politicaoptima(a):
    rows, columns = b.shape
    decision = np.zeros((2,columns-1))
    for i in range(3):
        for j in range(2):
            if a[i,j,0] > a[i,j,1]:
                decision[j,i]=1
    return decision

In [88]:
policy=politicaoptima(a)
print(policy)

[[1. 1. 0.]
 [1. 1. 1.]]


Notemos que para este caso, tenemos que en las etapas 1 y 2 conviene mandar a la niña al colegio. Para la etapa 3 (viernes) conviene no mandarla si está enferma (primera fila) y mandarla si está sana (segunda fila).